In [2]:
import torch
from PIL import Image
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.metrics import roc_auc_score
from torch.nn.functional import relu
from torchvision.io import read_image
import matplotlib.pyplot as plt

In [3]:
DATA_PATH = Path('/') / 'home' / 'user' / 'data'
TRAIN_DATA_PATH = DATA_PATH / 'ML' / 'train'
TEST_DATA_PATH = DATA_PATH / 'test'
TARGETS = ['target_0', 'target_1']

In [30]:
torch_device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

## Define the dataset and data loader

In [31]:
class CamelyonDataset(torch.utils.data.Dataset):

    def __init__(self, data_path, targets, transform=None):
        samples = list()
        labels = list()
        self.transform = transform
        for target in targets:
            data_paths = list((data_path / target).glob('*.jpg'))
            label = int(target.split('_')[-1])
            assert len(data_paths) > 0, f'Wrong data path: {data_path / target}'
            samples.extend(data_paths)
            labels.extend([label]*len(data_paths))
        self.samples = pd.DataFrame({'filepath': samples, 'label': labels})

    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
    
        img_path = self.samples.iloc[idx, 0]
        image = read_image(str(img_path.resolve())).to(torch_device)
        image = (1.0 / 255.0) * image
        
        label = self.samples.iloc[idx, 1]
        sample = image, label
        
        if self.transform:
            sample = self.transform(sample)
       
        return sample

In [33]:
dataset = CamelyonDataset(
    data_path=TRAIN_DATA_PATH,
    targets=TARGETS,
)
test_dataset = CamelyonDataset(
    data_path=TEST_DATA_PATH,
    targets=TARGETS,
)

In [34]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True, num_workers=0)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

In [35]:
# Evaluation code
def evaluate(model, loader):
    model.eval()

    all_labels = None
    all_preds = None

    for batch in loader:
        inputs, labels = batch
        labels = labels.to(torch_device).float().cpu().detach().numpy().tolist()
        preds = model(inputs).cpu().detach().numpy().tolist()

        if all_labels is None:
            all_labels = labels
            all_preds = preds
        else:
            all_labels.extend(labels)
            all_preds.extend(preds)

    return roc_auc_score(all_labels, all_preds)

## Define the model

In [ ]:
class TorchModel(torch.nn.Module):
    
    def __init__(
        self,
    ):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(3, 6, kernel_size=3)
        self.pool = torch.nn.MaxPool2d(2, 2)
        self.conv2 = torch.nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = torch.nn.Linear(16 * 53 * 53, 120)
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 1)
        
    def forward(self, x: torch.Tensor):
        x = self.conv1(x)
        x = self.pool(relu(x))
        x = self.pool(relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = relu(self.fc1(x))
        x = relu(self.fc2(x))
        x = self.fc3(x)
        return x.squeeze()
        

## Train and evaluate the model

In [37]:
n_epochs = 15

model = TorchModel().to(torch_device)

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

test_perfs = list()
train_perfs = list()

for epoch in range(n_epochs):
    model.train()
    for batch in dataloader:
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = batch
        labels = labels.to(torch_device).float()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    train_score = evaluate(model, loader=dataloader)
    train_perfs.append(train_score)

    test_score = evaluate(model, loader=test_dataloader)
    test_perfs.append(test_score)
    print(f'Epoch {epoch+1}/{n_epochs} - Train AUC: {train_score} - Test AUC: {test_score}')

Epoch 1/15 - Train AUC: 0.8861211309588095 - Test AUC: 0.7549141591982726
Epoch 2/15 - Train AUC: 0.9254097337720467 - Test AUC: 0.7691052549905185
Epoch 3/15 - Train AUC: 0.9505323519082509 - Test AUC: 0.832159748507468
Epoch 4/15 - Train AUC: 0.9915491284532476 - Test AUC: 0.7810508817465746
Epoch 5/15 - Train AUC: 0.9989866154034458 - Test AUC: 0.7728466964473817
Epoch 6/15 - Train AUC: 0.9998540022191663 - Test AUC: 0.7632065074073068
Epoch 7/15 - Train AUC: 0.9996284056482341 - Test AUC: 0.684168123088848
Epoch 8/15 - Train AUC: 0.9997896031980313 - Test AUC: 0.7679488440309739
Epoch 9/15 - Train AUC: 0.9999856002188767 - Test AUC: 0.7297199294830978
Epoch 10/15 - Train AUC: 0.9999806002948756 - Test AUC: 0.7354321699968863
Epoch 11/15 - Train AUC: 0.9997366040036192 - Test AUC: 0.6545714538567301
Epoch 12/15 - Train AUC: 0.9999988000182397 - Test AUC: 0.7164610333923734
Epoch 13/15 - Train AUC: 0.9999994000091198 - Test AUC: 0.7029758002881853
Epoch 14/15 - Train AUC: 1.0 - Test 

## Display the results

In [40]:
%matplotlib inline

In [1]:
# Plot the results
plt.title('ROC-AUC')
plt.plot(list(range(n_epochs)), train_perfs,label = 'Train AUC')
plt.plot(list(range(n_epochs)), test_perfs, label = 'Test AUC')
plt.legend(loc = 'lower right')
plt.ylabel('AUC')
plt.xlabel('epochs')
plt.show()

NameError: name 'plt' is not defined

## Kaggle submission